In [2]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.0

In [3]:
# Parametros

g = 5

# Costos

a = [3, 4.05, 4.05, 3.99, 3.88]
b = [20, 18.07, 15.55, 19.21, 26.18]
c = [100, 98.87, 104,26, 107.21, 95.31]

# Emisiones

d = [2, 3.82, 5.01, 1.10, 3.55]
e = [-5, -4.24, -2.15, -3.99, -6.88]
f = [3, 6.09, 5.69, 6.20, 5.57]

# Potencias

Pmin = [28, 90, 68, 76, 19]
Pmax = [206, 284, 189, 266, 53]


In [4]:
model = gp.Model('Generación Térmica')

# Variable de desición
potgen = model.addVars(g, vtype=GRB.CONTINUOUS, name="potgen")

Academic license - for non-commercial use only - expires 2021-05-28
Using license file C:\Users\56982\gurobi.lic


In [5]:
# Restricción de potencias

p_min = model.addConstrs((potgen[type] >= Pmin[type])
                              for type in range(g))

p_max = model.addConstrs((potgen[type] <= Pmax[type])
                              for type in range(g))

In [6]:
# Satisfacer la demanda

demanda = model.addConstr(gp.quicksum(potgen[type]  for type in range(g)) >= 400)

In [7]:
# Minimizar costos

costos = gp.quicksum(a[type]*potgen[type]*potgen[type]+b[type]*potgen[type]+c[type]
                    for type in range(g))

# emisiones = gp.quicksum(d[type]*potgen[type]+e[type]*potgen[type]+f[type]*potgen[type]
                  #  for type in range(g))



In [8]:
# Restricción de emisiones

# r_emisones = model.addConstr(emisiones <= 90000)

In [9]:
model.setObjective(costos) #+ emisiones)

In [10]:
model.write('cgk3_1.lp')
model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 11 rows, 5 columns and 15 nonzeros
Model fingerprint: 0x58dd521b
Model has 5 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 3e+01]
  QObjective range [6e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 4e+02]
Presolve removed 10 rows and 0 columns
Presolve time: 0.01s
Presolved: 1 rows, 5 columns, 5 nonzeros
Presolved model has 5 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.34599244e+06 -3.96232884e+06  7.95e+02 4.99e+02  1.30e+06     0s
   1   3.54147250e+05 -1.42369431e+

In [11]:
rows = ["Potencia" + str(t) for t in range(g)]
units = pd.DataFrame(columns=range(1), index=rows, data=0.0)

for t in range(g):
            units.loc["Potencia"+str(t)] = potgen[t].x
units

,0
Potencia0,102.844226
Potencia1,90.000000
Potencia2,76.730291
Potencia3,77.425483
Potencia4,53.000000


In [12]:
rows = ["Emisiones" + str(t) for t in range(g)]
units = pd.DataFrame(columns=range(1), index=rows, data=0.0)

for t in range(g):
            units.loc["Emisiones"+str(t)] = d[t]*potgen[t].x*potgen[t].x+e[t]*potgen[t].x+f[t]
units

,0
Emisiones0,20642.648501
Emisiones1,30566.490000
Emisiones2,29337.282914
Emisiones3,6291.448313
Emisiones4,9612.880000
